# Intializing Spark Session

In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession 
spark = SparkSession.builder.getOrCreate()

# creating dataframe

In [2]:
myRange = spark.range(1000).toDF("number")
myRange.show()

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
|     5|
|     6|
|     7|
|     8|
|     9|
|    10|
|    11|
|    12|
|    13|
|    14|
|    15|
|    16|
|    17|
|    18|
|    19|
+------+
only showing top 20 rows



# where

In [3]:
# doing transformation and action
divisBy2 = myRange.where("number % 2 = 0")
divisBy2.show()

+------+
|number|
+------+
|     0|
|     2|
|     4|
|     6|
|     8|
|    10|
|    12|
|    14|
|    16|
|    18|
|    20|
|    22|
|    24|
|    26|
|    28|
|    30|
|    32|
|    34|
|    36|
|    38|
+------+
only showing top 20 rows



# csv read

In [4]:
flightData2015 = spark\
.read\
.option("inferSchema", "true")\
.option("header", "true")\
.csv(r"C:\Users\Admin\Documents\spark\Spark-The-Definitive-Guide-master\data\flight-data\csv\2015-summary.csv")

In [5]:
flightData2015.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [6]:
flightData2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

# explain

In [7]:
# to see how spark will execute the query
# sort is a wide dependency transformation
flightData2015.sort("count").explain()

== Physical Plan ==
*(1) Sort [count#32 ASC NULLS FIRST], true, 0
+- *(1) Project [DEST_COUNTRY_NAME#30, ORIGIN_COUNTRY_NAME#31, count#32]
   +- BatchScan[DEST_COUNTRY_NAME#30, ORIGIN_COUNTRY_NAME#31, count#32] CSVScan Location: InMemoryFileIndex[file:/C:/Users/Admin/Documents/spark/Spark-The-Definitive-Guide-master/data/fli..., ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [8]:
# setting number of partitions upto 5
spark.conf.set("spark.sql.shuffle.partitions", "5")
flightData2015.sort("count").take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [9]:
# create table or view for spark sql
flightData2015.createOrReplaceTempView("flight_data_2015")

In [10]:
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")
# above logic in data frame
dataFrameWay = flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.count()

In [11]:
# both generates same physical plan so it doesn't matter which approach you are using
print(sqlWay.explain())
print(dataFrameWay.explain())

== Physical Plan ==
*(1) HashAggregate(keys=[DEST_COUNTRY_NAME#30], functions=[count(1)])
+- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#30], functions=[partial_count(1)])
   +- *(1) Project [DEST_COUNTRY_NAME#30]
      +- BatchScan[DEST_COUNTRY_NAME#30] CSVScan Location: InMemoryFileIndex[file:/C:/Users/Admin/Documents/spark/Spark-The-Definitive-Guide-master/data/fli..., ReadSchema: struct<DEST_COUNTRY_NAME:string>


None
== Physical Plan ==
*(1) HashAggregate(keys=[DEST_COUNTRY_NAME#30], functions=[count(1)])
+- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#30], functions=[partial_count(1)])
   +- *(1) Project [DEST_COUNTRY_NAME#30]
      +- BatchScan[DEST_COUNTRY_NAME#30] CSVScan Location: InMemoryFileIndex[file:/C:/Users/Admin/Documents/spark/Spark-The-Definitive-Guide-master/data/fli..., ReadSchema: struct<DEST_COUNTRY_NAME:string>


None


In [12]:
spark.sql("SELECT max(count) from flight_data_2015").take(1)

[Row(max(count)=370002)]

In [13]:
from pyspark.sql.functions import max
flightData2015.select(max("count")).take(1)

[Row(max(count)=370002)]

In [14]:
# finding top 5 destinations 
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")
maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [15]:
from pyspark.sql.functions import desc
flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.sum("count")\
.withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total"))\
.limit(5)\
.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



# STRUCTURED STREAMING EXAMPLE

In [16]:
staticDataFrame = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load(r"C:\Users\Admin\Documents\spark\Spark-The-Definitive-Guide-master\data\retail-data\by-day\*.csv")
staticDataFrame.createOrReplaceTempView("retail_data")
staticSchema = staticDataFrame.schema

In [17]:
# batch code
from pyspark.sql.functions import window, column, desc, col
staticDataFrame\
.selectExpr("CustomerId","(UnitPrice * Quantity) as total_cost","InvoiceDate")\
.groupBy(col("CustomerId"),window(col("InvoiceDate"),"1 day"))\
.sum("total_cost")\
.show()

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   14075.0|[2011-12-05 05:00...|316.78000000000003|
|   18180.0|[2011-12-05 05:00...|            310.73|
|   15358.0|[2011-12-05 05:00...| 830.0600000000003|
|   15392.0|[2011-12-05 05:00...|304.40999999999997|
|   15290.0|[2011-12-05 05:00...|263.02000000000004|
|   16811.0|[2011-12-05 05:00...|             232.3|
|   12748.0|[2011-12-05 05:00...| 363.7899999999999|
|   16500.0|[2011-12-05 05:00...| 52.74000000000001|
|   16873.0|[2011-12-05 05:00...|1854.8300000000002|
|   14060.0|[2011-12-05 05:00...|297.47999999999996|
|   14649.0|[2011-12-05 05:00...| 513.9899999999998|
|   16904.0|[2011-12-05 05:00...| 349.0200000000001|
|   17857.0|[2011-12-05 05:00...|            2979.6|
|   14083.0|[2011-12-05 05:00...| 446.5700000000001|
|   14777.0|[2011-12-05 05:00...|             -2.95|
|   16684.0|[2011-12-05 05:00...| 5401.9799999

In [18]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [19]:
streamingDataFrame = spark.readStream\
.schema(staticSchema)\
.option("maxFilesPerTrigger", 1)\
.format("csv")\
.option("header", "true")\
.load(r"C:\Users\Admin\Documents\spark\Spark-The-Definitive-Guide-master\data\retail-data\by-day\*.csv")

In [20]:
streamingDataFrame.isStreaming

True

In [21]:
purchaseByCustomerPerHour = streamingDataFrame\
.selectExpr(
"CustomerId",
"(UnitPrice * Quantity) as total_cost",
"InvoiceDate")\
.groupBy(
col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
.sum("total_cost")

In [22]:
# performing action on stream
"""purchaseByCustomerPerHour.writeStream\
.format('memory')\
.queryName('customer_purchases')\
.outputMode('complete')\
.start()"""

"purchaseByCustomerPerHour.writeStream.format('memory').queryName('customer_purchases').outputMode('complete').start()"

In [23]:
# spark.sql('SELECT * FROM customer_purchases ORDER BY `sum(total_cost)` DESC').show(5)

In [24]:
staticDataFrame.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [25]:
from pyspark.sql.functions import date_format, col
preppedDataFrame = staticDataFrame\
.na.fill(0)\
.withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE"))\
.coalesce(5)

In [26]:
trainDataFrame = preppedDataFrame\
.where("InvoiceDate < '2011-07-01'")
testDataFrame = preppedDataFrame\
.where("InvoiceDate >= '2011-07-01'")

In [27]:
trainDataFrame.count()
testDataFrame.count()

296006

In [28]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()\
.setInputCol("day_of_week")\
.setOutputCol("day_of_week_index")

In [29]:
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder()\
.setInputCol("day_of_week_index")\
.setOutputCol("day_of_week_encoded")

In [30]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler()\
.setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"])\
.setOutputCol("features")

In [31]:
from pyspark.ml import Pipeline
transformationPipeline = Pipeline()\
.setStages([indexer, encoder, vectorAssembler])

In [32]:
fittedPipeline = transformationPipeline.fit(trainDataFrame)

In [33]:
transformedTraining = fittedPipeline.transform(trainDataFrame)

In [34]:
transformedTraining.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+-----------------+-------------------+--------------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|day_of_week|day_of_week_index|day_of_week_encoded|            features|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+-----------------+-------------------+--------------------+
|   537226|    22811|SET OF 6 T-LIGHTS...|       6|2010-12-06 08:34:00|     2.95|   15987.0|United Kingdom|     Monday|              2.0|      (5,[2],[1.0])|(7,[0,1,4],[2.95,...|
|   537226|    21713|CITRONELLA CANDLE...|       8|2010-12-06 08:34:00|      2.1|   15987.0|United Kingdom|     Monday|              2.0|      (5,[2],[1.0])|(7,[0,1,4],[2.1,8...|
|   537226|    22927|GREEN GIANT GARDE...|       2|2010-12-06 08:34:00|     5.95|   15987.0|United Kingdo

In [35]:
transformedTraining.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string, day_of_week: string, day_of_week_index: double, day_of_week_encoded: vector, features: vector]

In [36]:
from pyspark.ml.clustering import KMeans
kmeans = KMeans().setK(20)

In [37]:
kmModel = kmeans.fit(transformedTraining)

In [38]:
# in Python
from pyspark.sql import Row
spark.sparkContext.parallelize([Row(1), Row(2), Row(3)]).toDF()

DataFrame[_1: bigint]

In [39]:
spark.range(2).collect()

[Row(id=0), Row(id=1)]

# how we instantiate, or declare, a column to be of a certain type.

In [40]:
# To work with the correct Python types, use the following
from pyspark.sql.types import *
b = ByteType()

# Basic Structured Operations

In [41]:
df = spark.read.format("json")\
    .load(r"C:\Users\Admin\Documents\spark\Spark-The-Definitive-Guide-master\data\flight-data\json\2015-summary.json")
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [42]:
spark.read.format("json")\
    .load(r"C:\Users\Admin\Documents\spark\Spark-The-Definitive-Guide-master\data\flight-data\json\2015-summary.json")\
    .schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

# Enforce Schema in Spark 

In [43]:
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema = StructType([
    StructField("DEST_COUNTRY_NAME",StringType(),True),
    StructField("ORIGIN_COUNTRY_NAME",StringType(),True),
    StructField("count",LongType(),False,metadata={"hello":"world"})
])
df = spark.read.format("json")\
    .schema(myManualSchema)\
    .load(r"C:\Users\Admin\Documents\spark\Spark-The-Definitive-Guide-master\data\flight-data\json\2015-summary.json")

In [44]:
df.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

# Creating Columns

In [45]:
from pyspark.sql.functions import col, column
col("someColumnName")
column("someColumnName")

Column<b'someColumnName'>

In [46]:
# to see columns
spark.read.format("json")\
    .load(r"C:\Users\Admin\Documents\spark\Spark-The-Definitive-Guide-master\data\flight-data\json\2015-summary.json")\
    .columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

In [47]:
df.first()

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

# Creating Rows

In [48]:
from pyspark.sql import Row
myRow = Row("Hello", None, 1, False)
myRow

<Row('Hello', None, 1, False)>

In [49]:
print(myRow[0])
print(myRow[2])

Hello
1


In [50]:
df.createOrReplaceTempView('dfTable')

In [51]:
# manually creating data frame
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, LongType
myManualSchema = StructType([
StructField("some", StringType(), True),
StructField("col", StringType(), True),
StructField("names", LongType(), False)
])
myRow = Row("Hello", None, 1)
myDf = spark.createDataFrame([myRow], myManualSchema)
myDf.show()

+-----+----+-----+
| some| col|names|
+-----+----+-----+
|Hello|null|    1|
+-----+----+-----+



In [52]:
df.select("DEST_COUNTRY_NAME").show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [53]:
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows



In [54]:
from pyspark.sql.functions import expr, col, column
df.select(
expr("DEST_COUNTRY_NAME"),
col("DEST_COUNTRY_NAME"),
column("DEST_COUNTRY_NAME"))\
.show(2)

+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|
|    United States|    United States|    United States|
+-----------------+-----------------+-----------------+
only showing top 2 rows



In [55]:
# this will result in error you cannot select with string 
df.select(col("DEST_COUNTRY_NAME"), "DEST_COUNTRY_NAME")

DataFrame[DEST_COUNTRY_NAME: string, DEST_COUNTRY_NAME: string]

In [56]:
df.select(expr("DEST_COUNTRY_NAME AS destination")).show(2)

+-------------+
|  destination|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows



In [57]:
df.select(expr("DEST_COUNTRY_NAME as destination").alias("DEST_COUNTRY_NAME"))\
.show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [58]:
# to use select with expr
df.selectExpr("DEST_COUNTRY_NAME as newColumnName", "DEST_COUNTRY_NAME").show(2)

+-------------+-----------------+
|newColumnName|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows



In [59]:
df.selectExpr(
"*", # all original columns
"(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry")\
.show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [60]:
df.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show(2)
# in sql
# SELECT avg(count), count(distinct(DEST_COUNTRY_NAME)) FROM dfTable LIMIT 2

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



In [61]:
from pyspark.sql.functions import lit
df.select(expr("*"), lit(1).alias("One")).show(2)
#SELECT *, 1 as One FROM dfTable LIMIT 2

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



# Adding Columns

In [62]:
df.withColumn("numberOne", lit(1)).show(2)
# SELECT *, 1 as numberOne FROM dfTable LIMIT 2

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [63]:
df.withColumn("withinCountry", expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME"))\
.show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [64]:
df.withColumn("Destination", expr("DEST_COUNTRY_NAME")).columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count', 'Destination']

# Renaming Columns

In [65]:
df.withColumnRenamed("DEST_COUNTRY_NAME", "dest").columns

['dest', 'ORIGIN_COUNTRY_NAME', 'count']

In [66]:
dfWithLongColName = df.withColumn(
"This Long Column-Name",
expr("ORIGIN_COUNTRY_NAME"))

# Reserved Characters and Keywords

In [67]:
"""
One thing that you might come across is reserved characters like spaces or dashes in column
names. Handling these means escaping column names appropriately. In Spark, we do this by
using backtick (`) characters
"""
dfWithLongColName = df.withColumn(
"This Long Column-Name",
expr("ORIGIN_COUNTRY_NAME"))

dfWithLongColName.show(3)

+-----------------+-------------------+-----+---------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|This Long Column-Name|
+-----------------+-------------------+-----+---------------------+
|    United States|            Romania|   15|              Romania|
|    United States|            Croatia|    1|              Croatia|
|    United States|            Ireland|  344|              Ireland|
+-----------------+-------------------+-----+---------------------+
only showing top 3 rows



In [68]:
dfWithLongColName.selectExpr(
"`This Long Column-Name`",
"`This Long Column-Name` as `new col`")\
.show(2)

+---------------------+-------+
|This Long Column-Name|new col|
+---------------------+-------+
|              Romania|Romania|
|              Croatia|Croatia|
+---------------------+-------+
only showing top 2 rows



In [69]:
dfWithLongColName.createOrReplaceTempView("dfTableLong")

In [70]:
dfWithLongColName.select(expr("`This Long Column-Name`")).columns

['This Long Column-Name']

# Removing Columns

In [71]:
df.drop("ORIGIN_COUNTRY_NAME").columns

['DEST_COUNTRY_NAME', 'count']

In [72]:
dfWithLongColName.drop("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME")

DataFrame[count: bigint, This Long Column-Name: string]

# Changing a Column’s Type (cast)

In [73]:
df.withColumn("count2", col("count").cast("long"))
# SELECT *, cast(count as long) AS count2 FROM dfTable

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint, count2: bigint]

# Filtering Rows

In [74]:
df.filter(col("count") < 2).show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [75]:
df.where("count < 2").show(2)
# SELECT * FROM dfTable WHERE count < 2 LIMIT 2

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [76]:
df.where(col("count") < 2).where(col("ORIGIN_COUNTRY_NAME") != "Croatia")\
.show(2)
# SELECT * FROM dfTable WHERE count < 2 AND ORIGIN_COUNTRY_NAME != "Croatia" LIMIT 2

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



# Getting Unique Rows

In [77]:
df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count()
# SELECT COUNT(DISTINCT(ORIGIN_COUNTRY_NAME, DEST_COUNTRY_NAME)) FROM dfTable

256

In [78]:
df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").count()

256

In [79]:
df.select("ORIGIN_COUNTRY_NAME").distinct().count()
# SELECT COUNT(DISTINCT ORIGIN_COUNTRY_NAME) FROM dfTable

125

# Random Samples

In [80]:
"""
Sometimes, you might just want to sample some random records from your DataFrame. You can
do this by using the sample method on a DataFrame, which makes it possible for you to specify
a fraction of rows to extract from a DataFrame and whether you’d like to sample with or without
replacement
"""
seed = 5
withReplacement = False
fraction = 0.5
df.sample(withReplacement, fraction, seed).count()

138

# Random Splits

In [81]:
"""
Random splits can be helpful when you need to break up your DataFrame into a random “splits”
of the original DataFrame. This is often used with machine learning algorithms to create training,
validation, and test sets. In this next example, we’ll split our DataFrame into two different
DataFrames by setting the weights by which we will split the DataFrame (these are the
arguments to the function). Because this method is designed to be randomized, we will also
specify a seed (just replace seed with a number of your choosing in the code block). It’s
important to note that if you don’t specify a proportion for each DataFrame that adds up to one,
they will be normalized so that they do
"""
dataFrames = df.randomSplit([0.25, 0.75], seed)
dataFrames[0].count() > dataFrames[1].count() # False

False

In [82]:
len(dataFrames)

2

# Concatenating and Appending Rows (Union)

In [83]:
"""
As you learned in the previous section, DataFrames are immutable. This means users cannot
append to DataFrames because that would be changing it. To append to a DataFrame, you must
union the original DataFrame along with the new DataFrame. This just concatenates the two
DataFramess. To union two DataFrames, you must be sure that they have the same schema and
number of columns; otherwise, the union will fail.
"""
from pyspark.sql import Row
schema = df.schema
newRows = [
Row("New Country", "Other Country", 5),
Row("New Country 2", "Other Country 3", 1)
]
parallelizedRows = spark.sparkContext.parallelize(newRows)
newDF = spark.createDataFrame(parallelizedRows, schema)
# in Python
df.union(newDF)\
.where("count = 1")\
.where(col("ORIGIN_COUNTRY_NAME") != "United States")\
.show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|    United States|          Gibraltar|    1|
|    United States|             Cyprus|    1|
|    United States|            Estonia|    1|
|    United States|          Lithuania|    1|
|    United States|           Bulgaria|    1|
|    United States|            Georgia|    1|
|    United States|            Bahrain|    1|
|    United States|   Papua New Guinea|    1|
|    United States|         Montenegro|    1|
|    United States|            Namibia|    1|
|    New Country 2|    Other Country 3|    1|
+-----------------+-------------------+-----+



In [84]:
df.where(col("DEST_COUNTRY_NAME")=="New Country 2").show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
+-----------------+-------------------+-----+



# Sorting Rows

In [85]:
df.sort("count").show(5)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows



In [86]:
df.orderBy("count", "DEST_COUNTRY_NAME").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
|         Djibouti|      United States|    1|
|        Indonesia|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [87]:
df.orderBy(col("count"), col("DEST_COUNTRY_NAME")).show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
|         Djibouti|      United States|    1|
|        Indonesia|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [88]:
from pyspark.sql.functions import desc, asc
df.orderBy(expr("count desc")).show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Moldova|      United States|    1|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [89]:
df.orderBy(col("count").desc(), col("DEST_COUNTRY_NAME").asc()).show(2)
# SELECT * FROM dfTable ORDER BY count DESC, DEST_COUNTRY_NAME ASC LIMIT 2

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
+-----------------+-------------------+------+
only showing top 2 rows



In [90]:
"""
An advanced tip is to use asc_nulls_first, desc_nulls_first, asc_nulls_last, or
desc_nulls_last to specify where you would like your null values to appear in an ordered
DataFrame.
"""
"""
For optimization purposes, it’s sometimes advisable to sort within each partition before another
set of transformations. You can use the sortWithinPartitions method to do this
"""
spark.read.format("json")\
.load(r"C:\Users\Admin\Documents\spark\Spark-The-Definitive-Guide-master\data\flight-data\json\*-summary.json")\
.sortWithinPartitions("count")

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

# Limit

In [91]:
df.limit(5).show()
# SELECT * FROM dfTable LIMIT 6

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+



In [92]:
df.orderBy(expr("count desc")).limit(6).show()
# SELECT * FROM dfTable ORDER BY count desc LIMIT 6

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
|             Moldova|      United States|    1|
+--------------------+-------------------+-----+



# Repartition and Coalesce

In [93]:
df.rdd.getNumPartitions() # 1

1

In [94]:
df.repartition(5)

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

In [95]:
"""
If you know that you’re going to be filtering by a certain column often, it can be worth
repartitioning based on that column:
"""
df.repartition(col("DEST_COUNTRY_NAME"))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

In [96]:
"""
You can optionally specify the number of partitions you would like, too:
"""
df.repartition(5, col("DEST_COUNTRY_NAME"))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

In [97]:
"""
Coalesce, on the other hand, will not incur a full shuffle and will try to combine partitions. This
operation will shuffle your data into five partitions based on the destination country name, and
then coalesce them (without a full shuffle):
"""
df.repartition(5, col("DEST_COUNTRY_NAME")).coalesce(2)

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

# Collecting Rows to the Driver

In [98]:
"""
Spark maintains the state of the cluster in the driver. There are times when you’ll want to collect some of your data to the 
driver in order to manipulate it on your local machine.

1-collect gets all data from the entire DataFrame.
2-take selects the first N rows.
3-show prints out a number of rows nicely.
"""
collectDF = df.limit(10)

In [99]:
collectDF.take(5) # take works with an Integer count

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)]

In [100]:
collectDF.show(5, False)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India              |62   |
+-----------------+-------------------+-----+
only showing top 5 rows



In [101]:
collectDF.collect()

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [102]:
"""
There’s an additional way of collecting rows to the driver in order to iterate over the entire
dataset. The method toLocalIterator collects partitions to the driver as an iterator. This
method allows you to iterate over the entire dataset partition-by-partition in a serial manner
"""
collectDF.toLocalIterator()

<generator object _local_iterator_from_socket.<locals>.PyLocalIterable.__iter__ at 0x000000000944FB30>

# Data Manipulation

In [103]:
df = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load(r"C:\Users\Admin\Documents\spark\Spark-The-Definitive-Guide-master\data\retail-data\by-day\2010-12-01.csv")
df.printSchema()


root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [104]:
df.createOrReplaceTempView("dfTable")

# Converting to Spark Types

In [105]:
"""
the lit function. This function converts a type in another language to its correspnding Spark representation.

-- in SQL
SELECT 5, "five", 5.0
"""
from pyspark.sql.functions import lit
df.select(lit(5), lit("five"), lit(5.0))

DataFrame[5: int, five: string, 5.0: double]

# Working with Booleans

In [106]:
"""
Boolean statements consist of four elements: and, or, true, and false.
"""
from pyspark.sql.functions import col
df.where(col("InvoiceNo") != 536365)\
.select("InvoiceNo", "Description")\
.show(5, False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows



In [107]:
"""
Another option—and probably the cleanest—is to specify the predicate as an expression in a
string. This is valid for Python or Scala. Note that this also gives you access to another way of
expressing “does not equal”:
"""
df.where("InvoiceNo = 536365").show(5, False)

+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |17850.0   |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |2010-12-01 08:26:00|2.75     |17850.0   |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
+---------+-----

In [108]:
df.where("InvoiceNo <> 536365").show(5, False)

+---------+---------+-----------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                  |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------+--------+-------------------+---------+----------+--------------+
|536366   |22633    |HAND WARMER UNION JACK       |6       |2010-12-01 08:28:00|1.85     |17850.0   |United Kingdom|
|536366   |22632    |HAND WARMER RED POLKA DOT    |6       |2010-12-01 08:28:00|1.85     |17850.0   |United Kingdom|
|536367   |84879    |ASSORTED COLOUR BIRD ORNAMENT|32      |2010-12-01 08:34:00|1.69     |13047.0   |United Kingdom|
|536367   |22745    |POPPY'S PLAYHOUSE BEDROOM    |6       |2010-12-01 08:34:00|2.1      |13047.0   |United Kingdom|
|536367   |22748    |POPPY'S PLAYHOUSE KITCHEN    |6       |2010-12-01 08:34:00|2.1      |13047.0   |United Kingdom|
+---------+---------+-----------------------------+--------+----

In [109]:
"""
-- in SQL
SELECT * FROM dfTable WHERE StockCode in ("DOT") AND(UnitPrice > 600 OR instr(Description, "POSTAGE") >= 1)
"""
from pyspark.sql.functions import instr
priceFilter = col("UnitPrice") > 600
descripFilter = instr(df.Description, "POSTAGE") >= 1
df.where(df.StockCode.isin("DOT")).where(priceFilter | descripFilter).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [110]:
"""
-- in SQL
SELECT UnitPrice, (StockCode = 'DOT' AND
(UnitPrice > 600 OR instr(Description, "POSTAGE") >= 1)) as isExpensive
FROM dfTable
WHERE (StockCode = 'DOT' AND
(UnitPrice > 600 OR instr(Description, "POSTAGE") >= 1))
"""
from pyspark.sql.functions import instr
DOTCodeFilter = col("StockCode") == "DOT"
priceFilter = col("UnitPrice") > 600
descripFilter = instr(col("Description"), "POSTAGE") >= 1
df.withColumn("isExpensive", DOTCodeFilter & (priceFilter | descripFilter))\
.where("isExpensive")\
.select("unitPrice", "isExpensive").show(5)

+---------+-----------+
|unitPrice|isExpensive|
+---------+-----------+
|   569.77|       true|
|   607.49|       true|
+---------+-----------+



In [111]:
# in Python
from pyspark.sql.functions import expr
df.withColumn("isExpensive", expr("NOT UnitPrice <= 250")).where("isExpensive").select("Description", "UnitPrice").show(5)

+--------------+---------+
|   Description|UnitPrice|
+--------------+---------+
|DOTCOM POSTAGE|   569.77|
|DOTCOM POSTAGE|   607.49|
+--------------+---------+



In [112]:
"""
One “gotcha” that can come up is if you’re working with null data when creating Boolean expressions.
If there is a null in your data, you’ll need to treat things a bit differently. Here’s how you can ensure
that you perform a null-safe equivalence test:
"""

df.where(col("Description").eqNullSafe("hello")).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
+---------+---------+-----------+--------+-----------+---------+----------+-------+



# Working with Numbers

In [113]:
"""
-- in SQL
SELECT customerId, (POWER((Quantity * UnitPrice), 2.0) + 5) as realQuantity
FROM dfTable
"""
# in Python
from pyspark.sql.functions import expr, pow
fabricatedQuantity = pow(col("Quantity") * col("UnitPrice"), 2) + 5
df.select(expr("CustomerId"), fabricatedQuantity.alias("realQuantity")).show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [114]:
# in Python
df.selectExpr(
"CustomerId",
"(POWER((Quantity * UnitPrice), 2.0) + 5) as realQuantity").show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [115]:
"""
-- in SQL
SELECT round(2.5), bround(2.5)
"""
# in Python
from pyspark.sql.functions import lit, round, bround
df.select(round(lit("2.5")), bround(lit("2.5"))).show(2)

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 2 rows



In [116]:
"""
Another numerical task is to compute the correlation of two columns. For example, we can see
the Pearson correlation coefficient for two columns to see if cheaper things are typically bought
in greater quantities. We can do this through a function as well as through the DataFrame
statistic methods:

-- in SQL
SELECT corr(Quantity, UnitPrice) FROM dfTable
"""
# in Python
from pyspark.sql.functions import corr
df.stat.corr("Quantity", "UnitPrice")
df.select(corr("Quantity", "UnitPrice")).show()

+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.04112314436835551|
+-------------------------+



In [117]:
"""
Another common task is to compute summary statistics for a column or set of columns. We can
use the describe method to achieve exactly this.
"""
df.describe().show()

+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
|summary|        InvoiceNo|         StockCode|         Description|          Quantity|        InvoiceDate|         UnitPrice|        CustomerID|       Country|
+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
|  count|             3108|              3108|                3098|              3108|               3108|              3108|              1968|          3108|
|   mean| 536516.684944841|27834.304044117645|                null| 8.627413127413128|               null| 4.151946589446603|15661.388719512195|          null|
| stddev|72.89447869788873|17407.897548583845|                null|26.371821677029203|               null|15.638659854603892|1854.4496996893627|          null|
|    min|           536365|             

In [118]:
"""
If you need these exact numbers, you can also perform this as an aggregation yourself by
importing the functions and applying them to the columns that you need:
"""
# in Python
from pyspark.sql.functions import count, mean, stddev_pop, min, max

# Quantiles

In [119]:
"""
There are a number of statistical functions available in the StatFunctions Package (accessible
using stat as we see in the code block below). These are DataFrame methods that you can use
to calculate a variety of different things. For instance, you can calculate either exact or
approximate quantiles of your data using the approxQuantile method:
"""
# in Python
colName = "UnitPrice"
quantileProbs = [0.5]
relError = 0.05
df.stat.approxQuantile("UnitPrice", quantileProbs, relError) # 2.51

[2.51]

# Pivot

In [120]:
df.stat.crosstab("StockCode", "Quantity").show(1)

+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|StockCode_Quantity| -1|-10|-12| -2|-24| -3| -4| -5| -6| -7|  1| 10|100| 11| 12|120|128| 13| 14|144| 15| 16| 17| 18| 19|192|  2| 20|200| 21|216| 22| 23| 24| 25|252| 27| 28|288|  3| 30| 32| 33| 34| 36|384|  4| 40|432| 47| 48|480|  5| 50| 56|  6| 60|600| 64|  7| 70| 72|  8| 80|  9| 96|
+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|             22578|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 

In [121]:
df.stat.freqItems(["StockCode", "Quantity"]).show()

+--------------------+--------------------+
| StockCode_freqItems|  Quantity_freqItems|
+--------------------+--------------------+
|[90214E, 20728, 2...|[200, 128, 23, 32...|
+--------------------+--------------------+



# monotonically_increasing_id

In [122]:
# in Python
from pyspark.sql.functions import monotonically_increasing_id
df.select(monotonically_increasing_id()).show(2)

+-----------------------------+
|monotonically_increasing_id()|
+-----------------------------+
|                            0|
|                            1|
+-----------------------------+
only showing top 2 rows



# Working with Strings

In [123]:
"""
-- in SQL
SELECT initcap(Description) FROM dfTable
"""
# in Python
from pyspark.sql.functions import initcap
df.select(initcap(col("Description"))).show()

+--------------------+
|initcap(Description)|
+--------------------+
|White Hanging Hea...|
| White Metal Lantern|
|Cream Cupid Heart...|
|Knitted Union Fla...|
|Red Woolly Hottie...|
|Set 7 Babushka Ne...|
|Glass Star Froste...|
|Hand Warmer Union...|
|Hand Warmer Red P...|
|Assorted Colour B...|
|Poppy's Playhouse...|
|Poppy's Playhouse...|
|Feltcraft Princes...|
|Ivory Knitted Mug...|
|Box Of 6 Assorted...|
|Box Of Vintage Ji...|
|Box Of Vintage Al...|
|Home Building Blo...|
|Love Building Blo...|
|Recipe Box With M...|
+--------------------+
only showing top 20 rows



In [124]:
"""
-- in SQL
SELECT Description, lower(Description), Upper(lower(Description)) FROM dfTable
"""

# in Python
from pyspark.sql.functions import lower, upper
df.select(col("Description"),
lower(col("Description")),
upper(lower(col("Description")))).show(2)

+--------------------+--------------------+-------------------------+
|         Description|  lower(Description)|upper(lower(Description))|
+--------------------+--------------------+-------------------------+
|WHITE HANGING HEA...|white hanging hea...|     WHITE HANGING HEA...|
| WHITE METAL LANTERN| white metal lantern|      WHITE METAL LANTERN|
+--------------------+--------------------+-------------------------+
only showing top 2 rows



In [125]:
"""
-- in SQL
SELECT
ltrim(' HELLLOOOO '),rtrim(' HELLLOOOO '),trim(' HELLLOOOO '),lpad('HELLOOOO ', 3, ' '),rpad('HELLOOOO ', 10, ' ')
FROM dfTable
"""
# in Python
from pyspark.sql.functions import lit, ltrim, rtrim, rpad, lpad, trim
df.select(
ltrim(lit(" HELLO ")).alias("ltrim"),
rtrim(lit(" HELLO ")).alias("rtrim"),
trim(lit(" HELLO ")).alias("trim"),
lpad(lit("HELLO"), 3, " ").alias("lp"),
rpad(lit("HELLO"), 10, " ").alias("rp")).show(2)

+------+------+-----+---+----------+
| ltrim| rtrim| trim| lp|        rp|
+------+------+-----+---+----------+
|HELLO | HELLO|HELLO|HEL|HELLO     |
|HELLO | HELLO|HELLO|HEL|HELLO     |
+------+------+-----+---+----------+
only showing top 2 rows



# Regular Expressions

In [126]:
"""
There are two key functions in Spark that you’ll need in
order to perform regular expression tasks: regexp_extract and regexp_replace. These
functions extract values and replace values, respectively.

-- in SQL
SELECT
regexp_replace(Description, 'BLACK|WHITE|RED|GREEN|BLUE', 'COLOR') as color_clean, Description
FROM dfTable
"""
# in Python
from pyspark.sql.functions import regexp_replace
regex_string = "BLACK|WHITE|RED|GREEN|BLUE"
df.select(
regexp_replace(col("Description"), regex_string, "COLOR").alias("color_clean"),
col("Description")).show(2)

+--------------------+--------------------+
|         color_clean|         Description|
+--------------------+--------------------+
|COLOR HANGING HEA...|WHITE HANGING HEA...|
| COLOR METAL LANTERN| WHITE METAL LANTERN|
+--------------------+--------------------+
only showing top 2 rows



In [127]:
"""
Another task might be to replace given characters with other characters. Building this as a
regular expression could be tedious, so Spark also provides the translate function to replace these
values. This is done at the character level and will replace all instances of a character with the
indexed character in the replacement string:

-- in SQL
SELECT translate(Description, 'LEET', '1337'), Description FROM dfTable
"""
# in Python
from pyspark.sql.functions import translate
df.select(translate(col("Description"), "LEET", "1337"),col("Description")).show(2)

+----------------------------------+--------------------+
|translate(Description, LEET, 1337)|         Description|
+----------------------------------+--------------------+
|              WHI73 HANGING H3A...|WHITE HANGING HEA...|
|               WHI73 M37A1 1AN73RN| WHITE METAL LANTERN|
+----------------------------------+--------------------+
only showing top 2 rows



In [128]:
"""
We can also perform something similar, like pulling out the first mentioned color:
-- in SQL
SELECT regexp_extract(Description, '(BLACK|WHITE|RED|GREEN|BLUE)', 1),
Description
FROM dfTable
"""
from pyspark.sql.functions import regexp_extract
extract_str = "(BLACK|WHITE|RED|GREEN|BLUE)"
df.select(
regexp_extract(col("Description"), extract_str, 1).alias("color_clean"),
col("Description")).show()

+-----------+--------------------+
|color_clean|         Description|
+-----------+--------------------+
|      WHITE|WHITE HANGING HEA...|
|      WHITE| WHITE METAL LANTERN|
|           |CREAM CUPID HEART...|
|           |KNITTED UNION FLA...|
|        RED|RED WOOLLY HOTTIE...|
|           |SET 7 BABUSHKA NE...|
|           |GLASS STAR FROSTE...|
|           |HAND WARMER UNION...|
|        RED|HAND WARMER RED P...|
|           |ASSORTED COLOUR B...|
|           |POPPY'S PLAYHOUSE...|
|           |POPPY'S PLAYHOUSE...|
|           |FELTCRAFT PRINCES...|
|           |IVORY KNITTED MUG...|
|           |BOX OF 6 ASSORTED...|
|           |BOX OF VINTAGE JI...|
|           |BOX OF VINTAGE AL...|
|           |HOME BUILDING BLO...|
|           |LOVE BUILDING BLO...|
|           |RECIPE BOX WITH M...|
+-----------+--------------------+
only showing top 20 rows



In [129]:
"""
Sometimes, rather than extracting values, we simply want to check for their existence. We can do
this with the contains method on each column. This will return a Boolean declaring whether the
value you specify is in the column’s string:

-- in SQL
SELECT Description FROM dfTable
WHERE instr(Description, 'BLACK') >= 1 OR instr(Description, 'WHITE') >= 1
"""
# in Python
from pyspark.sql.functions import instr
containsBlack = instr(col("Description"), "BLACK") >= 1
containsWhite = instr(col("Description"), "WHITE") >= 1
df.withColumn("hasSimpleColor", containsBlack | containsWhite)\
.where("hasSimpleColor")\
.select("Description").show(3, False)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



In [130]:
"""
locate function locates the position of the first occurrence of substr in a string column, after position pos.
"""
# in Python
from pyspark.sql.functions import expr, locate
simpleColors = ["black", "white", "red", "green", "blue"]
def color_locator(column, color_string):
    return locate(color_string.upper(), column)\
    .cast("boolean")\
    .alias("is_" + color_string)

In [131]:
selectedColumns = [color_locator(df.Description, c) for c in simpleColors]
selectedColumns.append(expr("*")) # has to a be Column type

In [132]:
for i in selectedColumns:
    print(i)

Column<b'CAST(locate(BLACK, Description, 1) AS BOOLEAN) AS `is_black`'>
Column<b'CAST(locate(WHITE, Description, 1) AS BOOLEAN) AS `is_white`'>
Column<b'CAST(locate(RED, Description, 1) AS BOOLEAN) AS `is_red`'>
Column<b'CAST(locate(GREEN, Description, 1) AS BOOLEAN) AS `is_green`'>
Column<b'CAST(locate(BLUE, Description, 1) AS BOOLEAN) AS `is_blue`'>
Column<b'unresolvedstar()'>


In [133]:
df.select(*selectedColumns).where(expr("is_white OR is_red"))\
.select("Description").show(3, False)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



# Working with Dates and Timestamps

In [134]:
from pyspark.sql.functions import current_date, current_timestamp
dateDF = spark.range(10)\
.withColumn("today", current_date())\
.withColumn("now", current_timestamp())

dateDF.show(5)

+---+----------+--------------------+
| id|     today|                 now|
+---+----------+--------------------+
|  0|2021-10-11|2021-10-11 17:28:...|
|  1|2021-10-11|2021-10-11 17:28:...|
|  2|2021-10-11|2021-10-11 17:28:...|
|  3|2021-10-11|2021-10-11 17:28:...|
|  4|2021-10-11|2021-10-11 17:28:...|
+---+----------+--------------------+
only showing top 5 rows



In [135]:
dateDF.createOrReplaceTempView("dateTable")
dateDF.printSchema()

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)



In [136]:
"""
Now that we have a simple DataFrame to work with, let’s add and subtract five days from today.
These functions take a column and then the number of days to either add or subtract as the
arguments:

-- in SQL
SELECT date_sub(today, 5), date_add(today, 5) FROM dateTable
"""
# in Python
from pyspark.sql.functions import date_add, date_sub
dateDF.select("today",date_sub(col("today"), 5), date_add(col("today"), 5)).show(1)

+----------+------------------+------------------+
|     today|date_sub(today, 5)|date_add(today, 5)|
+----------+------------------+------------------+
|2021-10-11|        2021-10-06|        2021-10-16|
+----------+------------------+------------------+
only showing top 1 row



In [137]:
"""
-- in SQL
SELECT to_date('2016-01-01'), months_between('2016-01-01', '2017-01-01'),
datediff('2016-01-01', '2017-01-01')
FROM dateTable
"""
from pyspark.sql.functions import datediff, months_between, to_date
dateDF.withColumn("week_ago", date_sub(col("today"), 7))\
.select(datediff(col("week_ago"), col("today"))).show(1)

+-------------------------+
|datediff(week_ago, today)|
+-------------------------+
|                       -7|
+-------------------------+
only showing top 1 row



In [138]:
dateDF.select(
to_date(lit("2016-01-01")).alias("start"),
to_date(lit("2017-05-22")).alias("end"))\
.select(months_between(col("start"), col("end"))).show(1)

+--------------------------------+
|months_between(start, end, true)|
+--------------------------------+
|                    -16.67741935|
+--------------------------------+
only showing top 1 row



In [139]:
"""
Notice that we introduced a new function: the to_date function. The to_date function allows
you to convert a string to a date, optionally with a specified format. We specify our format in the
Java SimpleDateFormat which will be important to reference if you use this function:
"""
from pyspark.sql.functions import to_date, lit
spark.range(5).withColumn("date", lit("2017-01-01"))\
.select(to_date(col("date"))).show(1)

+---------------+
|to_date(`date`)|
+---------------+
|     2017-01-01|
+---------------+
only showing top 1 row



In [140]:
"""
Spark will not throw an error if it cannot parse the date; rather, it will just return null. This can
be a bit tricky in larger pipelines because you might be expecting your data in one format and
getting it in another
"""
dateDF.select(to_date(lit("2016-20-12")),to_date(lit("2017-12-11"))).show(1)

+---------------------+---------------------+
|to_date('2016-20-12')|to_date('2017-12-11')|
+---------------------+---------------------+
|                 null|           2017-12-11|
+---------------------+---------------------+
only showing top 1 row



In [141]:
# its very very important to specifiy date format in spark because if you don't you can get bugs what become difficult to find
# in Python
"""
-- in SQL
SELECT to_date(date, 'yyyy-dd-MM'), to_date(date2, 'yyyy-dd-MM'), to_date(date)
FROM dateTable2
"""
from pyspark.sql.functions import to_date
dateFormat = "yyyy-dd-MM"
cleanDateDF = spark.range(1).select(
to_date(lit("2017-12-11"), dateFormat).alias("date"),
to_date(lit("2017-20-12"), dateFormat).alias("date2"))

cleanDateDF.show(5)

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



In [142]:
cleanDateDF.createOrReplaceTempView("dateTable2")

In [143]:
"""to_timestamp, which always requires a format to be specified:
-- in SQL
SELECT to_timestamp(date, 'yyyy-dd-MM'), to_timestamp(date2, 'yyyy-dd-MM')
FROM dateTable2
"""
# in Python
from pyspark.sql.functions import to_timestamp
cleanDateDF.select(to_timestamp(col("date"), dateFormat)).show()

+----------------------------------+
|to_timestamp(`date`, 'yyyy-dd-MM')|
+----------------------------------+
|               2017-11-12 00:00:00|
+----------------------------------+



In [144]:
"""
After we have our date or timestamp in the correct format and type, comparing between them is
actually quite easy. We just need to be sure to either use a date/timestamp type or specify our
string according to the right format of yyyy-MM-dd if we’re comparing a date:
"""
cleanDateDF.filter(col("date2") > lit("2017-12-12")).show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



In [145]:
cleanDateDF.filter(col("date2") > "'2017-12-12'").show()

+----+-----+
|date|date2|
+----+-----+
+----+-----+



# Working with Nulls in Data

In [146]:
# in Python
from pyspark.sql.functions import coalesce
df.select(coalesce(col("Description"), col("CustomerId"))).show()

+---------------------------------+
|coalesce(Description, CustomerId)|
+---------------------------------+
|             WHITE HANGING HEA...|
|              WHITE METAL LANTERN|
|             CREAM CUPID HEART...|
|             KNITTED UNION FLA...|
|             RED WOOLLY HOTTIE...|
|             SET 7 BABUSHKA NE...|
|             GLASS STAR FROSTE...|
|             HAND WARMER UNION...|
|             HAND WARMER RED P...|
|             ASSORTED COLOUR B...|
|             POPPY'S PLAYHOUSE...|
|             POPPY'S PLAYHOUSE...|
|             FELTCRAFT PRINCES...|
|             IVORY KNITTED MUG...|
|             BOX OF 6 ASSORTED...|
|             BOX OF VINTAGE JI...|
|             BOX OF VINTAGE AL...|
|             HOME BUILDING BLO...|
|             LOVE BUILDING BLO...|
|             RECIPE BOX WITH M...|
+---------------------------------+
only showing top 20 rows



In [147]:
df.na.drop("all", subset=["StockCode", "InvoiceNo"])

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

In [148]:
"""
Using the fill function, you can fill one or more columns with a set of values. This can be done
by specifying a map—that is a particular value and a set of columns.
For example, to fill all null values in columns of type String, you might specify the following:
"""
df.na.fill("All Null values become this string")

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

In [149]:
"""We could do the same for columns of type Integer by using df.na.fill(5:Integer), or for
Doubles df.na.fill(5:Double). To specify columns, we just pass in an array of column names
like we did in the previous example:"""
# in Python
df.na.fill("all", subset=["StockCode", "InvoiceNo"])

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

In [150]:
"""We can also do this with with a Scala Map, where the key is the column name and the value is the
value we would like to use to fill null values:"""
# in Python
fill_cols_vals = {"StockCode": 5, "Description" : "No Value"}
df.na.fill(fill_cols_vals)

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

In [151]:
"""In addition to replacing null values like we did with drop and fill, there are more flexible
options that you can use with more than just null values. Probably the most common use case is
to replace all values in a certain column according to their current value. The only requirement is
that this value be the same type as the original value:"""
# in Python
df.na.replace([""], ["UNKNOWN"], "Description")

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

# Working with Complex Types

In [152]:
df.selectExpr("(Description, InvoiceNo) as complex", "*").show(5)

+--------------------+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|             complex|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+--------------------+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|[WHITE HANGING HE...|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|[WHITE METAL LANT...|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|[CREAM CUPID HEAR...|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|[KNITTED UNION FL...|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|[RED WOOLLY HOTTI...|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     

In [153]:
df.selectExpr("struct(Description, InvoiceNo) as complex", "*").show(4)

+--------------------+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|             complex|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+--------------------+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|[WHITE HANGING HE...|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|[WHITE METAL LANT...|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|[CREAM CUPID HEAR...|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|[KNITTED UNION FL...|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+--------------------+---------+---------+--------------------+--------+-------------------+-----

In [154]:
# in Python
from pyspark.sql.functions import struct
complexDF = df.select(struct("Description", "InvoiceNo").alias("complex"))
complexDF.createOrReplaceTempView("complexDF")

In [155]:
"""We now have a DataFrame with a column complex. We can query it just as we might another
DataFrame, the only difference is that we use a dot syntax to do so, or the column method
getField:"""
complexDF.select("complex.Description").show(5)

+--------------------+
|         Description|
+--------------------+
|WHITE HANGING HEA...|
| WHITE METAL LANTERN|
|CREAM CUPID HEART...|
|KNITTED UNION FLA...|
|RED WOOLLY HOTTIE...|
+--------------------+
only showing top 5 rows



In [156]:
complexDF.select(col("complex").getField("Description")).show(5)

+--------------------+
| complex.Description|
+--------------------+
|WHITE HANGING HEA...|
| WHITE METAL LANTERN|
|CREAM CUPID HEART...|
|KNITTED UNION FLA...|
|RED WOOLLY HOTTIE...|
+--------------------+
only showing top 5 rows



In [157]:
complexDF.select("complex.*").show(5)

+--------------------+---------+
|         Description|InvoiceNo|
+--------------------+---------+
|WHITE HANGING HEA...|   536365|
| WHITE METAL LANTERN|   536365|
|CREAM CUPID HEART...|   536365|
|KNITTED UNION FLA...|   536365|
|RED WOOLLY HOTTIE...|   536365|
+--------------------+---------+
only showing top 5 rows



In [158]:
# ARRAYS
"""To define arrays, let’s work through a use case. With our current data, our objective is to take
every single word in our Description column and convert that into a row in our DataFrame.
The first task is to turn our Description column into a complex type, an array.
-- in SQL
SELECT split(Description, ' ') FROM dfTable
"""
# in Python
from pyspark.sql.functions import split
df.select(split(col("Description"), " ")).show(2)

+-------------------------+
|split(Description,  , -1)|
+-------------------------+
|     [WHITE, HANGING, ...|
|     [WHITE, METAL, LA...|
+-------------------------+
only showing top 2 rows



In [159]:
"""This is quite powerful because Spark allows us to manipulate this complex type as another
column. We can also query the values of the array using Python-like syntax:
-- in SQL
SELECT split(Description, ' ')[0] FROM dfTable
"""
# in Python
df.select(split(col("Description"), " ").alias("array_col"))\
.selectExpr("array_col[0]").show(2)

+------------+
|array_col[0]|
+------------+
|       WHITE|
|       WHITE|
+------------+
only showing top 2 rows



In [160]:
"""We can determine the array’s length by querying for its size:"""
# in Python
from pyspark.sql.functions import size
df.select(size(split(col("Description"), " "))).show(2) # shows 5 and 3

+-------------------------------+
|size(split(Description,  , -1))|
+-------------------------------+
|                              5|
|                              3|
+-------------------------------+
only showing top 2 rows



In [161]:
"""We can also see whether this array contains a value:
-- in SQL
SELECT array_contains(split(Description, ' '), 'WHITE') FROM dfTable"""
from pyspark.sql.functions import array_contains
df.select(array_contains(split(col("Description"), " "), "WHITE")).show(2)

+------------------------------------------------+
|array_contains(split(Description,  , -1), WHITE)|
+------------------------------------------------+
|                                            true|
|                                            true|
+------------------------------------------------+
only showing top 2 rows



In [162]:
"""To convert a complex type into a set of rows
(one per value in our array), we need to use the explode function."""
# in Python
from pyspark.sql.functions import split, explode
df.withColumn("splitted", split(col("Description"), " "))\
.withColumn("exploded", explode(col("splitted")))\
.select("Description", "InvoiceNo", "exploded").show()

+--------------------+---------+--------+
|         Description|InvoiceNo|exploded|
+--------------------+---------+--------+
|WHITE HANGING HEA...|   536365|   WHITE|
|WHITE HANGING HEA...|   536365| HANGING|
|WHITE HANGING HEA...|   536365|   HEART|
|WHITE HANGING HEA...|   536365| T-LIGHT|
|WHITE HANGING HEA...|   536365|  HOLDER|
| WHITE METAL LANTERN|   536365|   WHITE|
| WHITE METAL LANTERN|   536365|   METAL|
| WHITE METAL LANTERN|   536365| LANTERN|
|CREAM CUPID HEART...|   536365|   CREAM|
|CREAM CUPID HEART...|   536365|   CUPID|
|CREAM CUPID HEART...|   536365|  HEARTS|
|CREAM CUPID HEART...|   536365|    COAT|
|CREAM CUPID HEART...|   536365|  HANGER|
|KNITTED UNION FLA...|   536365| KNITTED|
|KNITTED UNION FLA...|   536365|   UNION|
|KNITTED UNION FLA...|   536365|    FLAG|
|KNITTED UNION FLA...|   536365|     HOT|
|KNITTED UNION FLA...|   536365|   WATER|
|KNITTED UNION FLA...|   536365|  BOTTLE|
|RED WOOLLY HOTTIE...|   536365|     RED|
+--------------------+---------+--

In [163]:
"""
Maps are created by using the map function and key-value pairs of columns. You then can select
them just like you might select from an array:"""
# in Python
from pyspark.sql.functions import create_map
df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map"))\
.show()

+--------------------+
|         complex_map|
+--------------------+
|[WHITE HANGING HE...|
|[WHITE METAL LANT...|
|[CREAM CUPID HEAR...|
|[KNITTED UNION FL...|
|[RED WOOLLY HOTTI...|
|[SET 7 BABUSHKA N...|
|[GLASS STAR FROST...|
|[HAND WARMER UNIO...|
|[HAND WARMER RED ...|
|[ASSORTED COLOUR ...|
|[POPPY'S PLAYHOUS...|
|[POPPY'S PLAYHOUS...|
|[FELTCRAFT PRINCE...|
|[IVORY KNITTED MU...|
|[BOX OF 6 ASSORTE...|
|[BOX OF VINTAGE J...|
|[BOX OF VINTAGE A...|
|[HOME BUILDING BL...|
|[LOVE BUILDING BL...|
|[RECIPE BOX WITH ...|
+--------------------+
only showing top 20 rows



In [164]:
"""# in Python
df.select(map(col("Description"), col("InvoiceNo")).alias("complex_map"))\
.selectExpr("complex_map['WHITE METAL LANTERN']").show(2)"""

'# in Python\ndf.select(map(col("Description"), col("InvoiceNo")).alias("complex_map")).selectExpr("complex_map[\'WHITE METAL LANTERN\']").show(2)'

# Working with JSON

In [165]:
# in Python
jsonDF = spark.range(1).selectExpr("""
'{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}' as jsonString""")

In [166]:
jsonDF.show()

+--------------------+
|          jsonString|
+--------------------+
|{"myJSONKey" : {"...|
+--------------------+



In [167]:
"""You can use the get_json_object to inline query a JSON object, be it a dictionary or array.
You can use json_tuple if this object has only one level of nesting:"""
from pyspark.sql.functions import get_json_object, json_tuple
jsonDF.select(
get_json_object(col("jsonString"), "$.myJSONKey.myJSONValue[1]").alias("column"),json_tuple(col("jsonString"), "myJSONKey")).show(2)

+------+--------------------+
|column|                  c0|
+------+--------------------+
|     2|{"myJSONValue":[1...|
+------+--------------------+



In [168]:
jsonDF.selectExpr(
"json_tuple(jsonString, '$.myJSONKey.myJSONValue[1]') as column").show(2)

+------+
|column|
+------+
|  null|
+------+



In [169]:
# in Python
from pyspark.sql.functions import from_json,to_json
from pyspark.sql.types import *
parseSchema = StructType((
StructField("InvoiceNo",StringType(),True),
StructField("Description",StringType(),True)))

df.selectExpr("(InvoiceNo, Description) as myStruct")\
.select(to_json(col("myStruct")).alias("newJSON"))\
.select(from_json(col("newJSON"), parseSchema), col("newJSON")).show(2)

+--------------------+--------------------+
|  from_json(newJSON)|             newJSON|
+--------------------+--------------------+
|[536365, WHITE HA...|{"InvoiceNo":"536...|
|[536365, WHITE ME...|{"InvoiceNo":"536...|
+--------------------+--------------------+
only showing top 2 rows



# UDFs

In [170]:
# make UDF
# in Python
udfExampleDF = spark.range(5).toDF("num")
def power3(double_value):
    return double_value ** 3
power3(2.0)

8.0

In [171]:
# register UDF
# in Python
from pyspark.sql.functions import udf
power3udf = udf(power3)

In [172]:
# use UDF
# in Python
from pyspark.sql.functions import col
udfExampleDF.select(power3udf(col("num"))).show(2)

+-----------+
|power3(num)|
+-----------+
|          0|
|          1|
+-----------+
only showing top 2 rows



In [173]:
# register udf 
# in Python
from pyspark.sql.types import IntegerType, DoubleType
spark.udf.register("power3py", power3, DoubleType())

<function __main__.power3(double_value)>

In [174]:
"""we are getting null here becuase there is mismatch in return type"""
# in Python
udfExampleDF.selectExpr("power3py(num)").show(2)
# registered via Python

+-------------+
|power3py(num)|
+-------------+
|         null|
|         null|
+-------------+
only showing top 2 rows



# AGGREGATION

In [175]:
# in Python
df = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load(r"C:\Users\Admin\Documents\spark\Spark-The-Definitive-Guide-master\data\retail-data\all\*.csv")\
.coalesce(5)
df.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: int, Country: string]

In [176]:
df.createOrReplaceTempView("dfTable")

In [177]:
df.select(count("StockCode")).show()

+----------------+
|count(StockCode)|
+----------------+
|          541909|
+----------------+



In [178]:
# in Python
from pyspark.sql.functions import countDistinct
df.select(countDistinct("StockCode")).show() # 4070

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



In [179]:
"""Often, we find ourselves working with large datasets and the exact distinct count is irrelevant.
There are times when an approximation to a certain degree of accuracy will work just fine, and
for that, you can use the approx_count_distinct function:

You will notice that approx_count_distinct took another parameter with which you can
specify the maximum estimation error allowed. In this case, we specified a rather large error and
thus receive an answer that is quite far off but does complete more quickly than countDistinct.
You will see much greater performance gains with larger datasets."""
# in Python
from pyspark.sql.functions import approx_count_distinct
df.select(approx_count_distinct("StockCode", 0.1)).show() # 3364

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+



In [180]:
# in Python
from pyspark.sql.functions import first, last
df.select(first("StockCode"), last("StockCode")).show()

+-----------------------+----------------------+
|first(StockCode, false)|last(StockCode, false)|
+-----------------------+----------------------+
|                 85123A|                 22138|
+-----------------------+----------------------+



In [181]:
from pyspark.sql.functions import min, max
df.select(min("Quantity"), max("Quantity")).show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



In [182]:
df.selectExpr("sum(INT(Quantity))").show() # 5176450

+--------------------------+
|sum(CAST(Quantity AS INT))|
+--------------------------+
|                   5176450|
+--------------------------+



In [183]:
spark.sql("SELECT SUM(Quantity) FROM dfTable").show()

+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+



In [184]:
# in Python
from pyspark.sql.functions import sum, count, avg, expr
df.select(
count("Quantity").alias("total_transactions"),
sum("Quantity").alias("total_purchases"),
avg("Quantity").alias("avg_purchases"),
expr("mean(Quantity)").alias("mean_purchases"))\
.selectExpr(
"total_purchases/total_transactions",
"avg_purchases",
"mean_purchases").show()

+--------------------------------------+----------------+----------------+
|(total_purchases / total_transactions)|   avg_purchases|  mean_purchases|
+--------------------------------------+----------------+----------------+
|                      9.55224954743324|9.55224954743324|9.55224954743324|
+--------------------------------------+----------------+----------------+



In [185]:
from pyspark.sql.functions import var_pop, stddev_pop
from pyspark.sql.functions import var_samp, stddev_samp
df.select(var_pop("Quantity"), var_samp("Quantity"),
stddev_pop("Quantity"), stddev_samp("Quantity")).show()

+-----------------+------------------+--------------------+---------------------+
|var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+-----------------+------------------+--------------------+---------------------+
|47559.30364660923| 47559.39140929892|  218.08095663447835|   218.08115785023455|
+-----------------+------------------+--------------------+---------------------+



In [186]:
spark.sql("""SELECT var_pop(Quantity), var_samp(Quantity),
stddev_pop(Quantity), stddev_samp(Quantity)
FROM dfTable""").show()

+---------------------------------+----------------------------------+------------------------------------+-------------------------------------+
|var_pop(CAST(Quantity AS DOUBLE))|var_samp(CAST(Quantity AS DOUBLE))|stddev_pop(CAST(Quantity AS DOUBLE))|stddev_samp(CAST(Quantity AS DOUBLE))|
+---------------------------------+----------------------------------+------------------------------------+-------------------------------------+
|                47559.30364660923|                 47559.39140929892|                  218.08095663447835|                   218.08115785023455|
+---------------------------------+----------------------------------+------------------------------------+-------------------------------------+



# skewness and kurtosis

In [187]:
from pyspark.sql.functions import skewness,kurtosis
df.select(skewness("Quantity"),kurtosis("Quantity")).show()

+--------------------+------------------+
|  skewness(Quantity)|kurtosis(Quantity)|
+--------------------+------------------+
|-0.26407557610528376|119768.05495530753|
+--------------------+------------------+



In [188]:
spark.sql("""SELECT skewness(Quantity), kurtosis(Quantity) FROM dfTable""").show()

+----------------------------------+----------------------------------+
|skewness(CAST(Quantity AS DOUBLE))|kurtosis(CAST(Quantity AS DOUBLE))|
+----------------------------------+----------------------------------+
|              -0.26407557610528376|                119768.05495530753|
+----------------------------------+----------------------------------+



# Covariance and Correlation

In [189]:
"""Correlation measures the Pearson correlation
coefficient, which is scaled between –1 and +1. The covariance is scaled according to the inputs
in the data.
Like the var function, covariance can be calculated either as the sample covariance or the
population covariance.
"""
from pyspark.sql.functions import corr,covar_pop,covar_samp
df.select(corr("InvoiceNo", "Quantity"),covar_pop("InvoiceNo", "Quantity"),covar_samp("InvoiceNo", "Quantity")).show()

+-------------------------+------------------------------+-------------------------------+
|corr(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|
+-------------------------+------------------------------+-------------------------------+
|     4.912186085640497E-4|            1052.7260778754955|             1052.7280543915997|
+-------------------------+------------------------------+-------------------------------+



In [190]:
spark.sql("""SELECT corr(InvoiceNo, Quantity), covar_samp(InvoiceNo, Quantity),
covar_pop(InvoiceNo, Quantity)
FROM dfTable""").show()

+---------------------------------------------------------+---------------------------------------------------------------+--------------------------------------------------------------+
|corr(CAST(InvoiceNo AS DOUBLE), CAST(Quantity AS DOUBLE))|covar_samp(CAST(InvoiceNo AS DOUBLE), CAST(Quantity AS DOUBLE))|covar_pop(CAST(InvoiceNo AS DOUBLE), CAST(Quantity AS DOUBLE))|
+---------------------------------------------------------+---------------------------------------------------------------+--------------------------------------------------------------+
|                                     4.912186085640497E-4|                                             1052.7280543915997|                                            1052.7260778754955|
+---------------------------------------------------------+---------------------------------------------------------------+--------------------------------------------------------------+



# Aggregating to Complex Types

In [191]:
"""
In Spark, you can perform aggregations not just of numerical values using formulas, you can also
perform them on complex types. For example, we can collect a list of values present in a given
column or only the unique values by collecting to a set."""
df.select("Country").show()

+--------------+
|       Country|
+--------------+
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
+--------------+
only showing top 20 rows



In [192]:
from pyspark.sql.functions import collect_set,collect_list
df.agg(collect_set("Country"),collect_list("Country")).show()

+--------------------+---------------------+
|collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
|[Portugal, Italy,...| [United Kingdom, ...|
+--------------------+---------------------+



In [193]:
df.groupby("InvoiceNo", "CustomerId").agg(collect_set("Country"),collect_list("Country")).show()

+---------+----------+--------------------+---------------------+
|InvoiceNo|CustomerId|collect_set(Country)|collect_list(Country)|
+---------+----------+--------------------+---------------------+
|   536366|     17850|    [United Kingdom]| [United Kingdom, ...|
|   536367|     13047|    [United Kingdom]| [United Kingdom, ...|
|   536369|     13047|    [United Kingdom]|     [United Kingdom]|
|   536376|     15291|    [United Kingdom]| [United Kingdom, ...|
|   536387|     16029|    [United Kingdom]| [United Kingdom, ...|
|   536392|     13705|    [United Kingdom]| [United Kingdom, ...|
|   536399|     17850|    [United Kingdom]| [United Kingdom, ...|
|   536403|     12791|       [Netherlands]| [Netherlands, Net...|
|   536405|     14045|    [United Kingdom]|     [United Kingdom]|
|   536415|     12838|    [United Kingdom]| [United Kingdom, ...|
|   536446|     15983|    [United Kingdom]| [United Kingdom, ...|
|   536463|     14849|    [United Kingdom]|     [United Kingdom]|
|   536464

In [194]:
spark.sql("""SELECT collect_set(Country), collect_set(Country) FROM dfTable""").show()

+--------------------+--------------------+
|collect_set(Country)|collect_set(Country)|
+--------------------+--------------------+
|[Portugal, Italy,...|[Portugal, Italy,...|
+--------------------+--------------------+



# Grouping

In [195]:
df.groupBy("InvoiceNo", "CustomerId").count().show()

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   536366|     17850|    2|
|   536367|     13047|   12|
|   536369|     13047|    1|
|   536376|     15291|    2|
|   536387|     16029|    5|
|  C536391|     17548|    7|
|   536392|     13705|   10|
|   536399|     17850|    2|
|   536403|     12791|    2|
|   536405|     14045|    1|
|   536415|     12838|   59|
|   536446|     15983|   32|
|   536463|     14849|    1|
|   536464|     17968|   85|
|   536500|     17377|   15|
|   536525|     14078|   13|
|   536529|     14237|    9|
|   536531|     15485|   23|
|   536532|     12433|   73|
|   536533|     16955|    6|
+---------+----------+-----+
only showing top 20 rows



In [196]:
spark.sql("""SELECT InvoiceNo,CustomerId,count(*) FROM dfTable GROUP BY InvoiceNo, CustomerId""").show()

+---------+----------+--------+
|InvoiceNo|CustomerId|count(1)|
+---------+----------+--------+
|   536366|     17850|       2|
|   536367|     13047|      12|
|   536369|     13047|       1|
|   536376|     15291|       2|
|   536387|     16029|       5|
|  C536391|     17548|       7|
|   536392|     13705|      10|
|   536399|     17850|       2|
|   536403|     12791|       2|
|   536405|     14045|       1|
|   536415|     12838|      59|
|   536446|     15983|      32|
|   536463|     14849|       1|
|   536464|     17968|      85|
|   536500|     17377|      15|
|   536525|     14078|      13|
|   536529|     14237|       9|
|   536531|     15485|      23|
|   536532|     12433|      73|
|   536533|     16955|       6|
+---------+----------+--------+
only showing top 20 rows



# Grouping with Expressions

In [197]:
from pyspark.sql.functions import count
df.groupby("InvoiceNo").agg(count("Quantity").alias("quan"),expr("count(Quantity)")).show()

+---------+----+---------------+
|InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
|   536370|  20|             20|
|   536380|   1|              1|
|   536384|  13|             13|
|   536387|   5|              5|
|   536397|   2|              2|
|   536405|   1|              1|
|   536407|   2|              2|
|   536463|   1|              1|
|   536500|  15|             15|
|   536522|  54|             54|
|   536523|  12|             12|
|   536536|   3|              3|
|   536538|  31|             31|
|   536542|  16|             16|
|   536555|   2|              2|
|   536561|  15|             15|
|   536573|   4|              4|
|   536579|   2|              2|
|   536580|   6|              6|
|   536582|  17|             17|
+---------+----+---------------+
only showing top 20 rows



# Grouping with Maps

In [198]:
"""Sometimes, it can be easier to specify your transformations as a series of Maps for which the key
is the column, and the value is the aggregation function (as a string) that you would like to
perform"""
df.groupBy("InvoiceNo").agg(expr("avg(Quantity)"),expr("stddev_pop(Quantity)")).show()

+---------+------------------+--------------------+
|InvoiceNo|     avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
|   536370|             22.45|   8.935742834258381|
|   536380|              24.0|                 0.0|
|   536384|14.615384615384615|  15.750645708563392|
|   536387|             288.0|  117.57550765359255|
|   536397|              30.0|                18.0|
|   536405|             128.0|                 0.0|
|   536407|               6.0|                 0.0|
|   536463|              12.0|                 0.0|
|   536500|               6.8|   4.019950248448356|
|   536522|1.5925925925925926|  1.6046058535136642|
|   536523| 9.333333333333334|   7.487025815072067|
|   536536|31.666666666666668|  34.373762603991366|
|   536538| 4.709677419354839|  3.7173833008743054|
|   536542|              24.5|    8.73212459828649|
|   536555|               1.0|                 0.0|
|   536561| 9.333333333333334|  2.9814239699997196|
|   536573| 

In [199]:
spark.sql("""SELECT InvoiceNo,avg(Quantity), stddev_pop(Quantity), InvoiceNo FROM dfTable
GROUP BY InvoiceNo""").show()

+---------+------------------+------------------------------------+---------+
|InvoiceNo|     avg(Quantity)|stddev_pop(CAST(Quantity AS DOUBLE))|InvoiceNo|
+---------+------------------+------------------------------------+---------+
|   536370|             22.45|                   8.935742834258381|   536370|
|   536380|              24.0|                                 0.0|   536380|
|   536384|14.615384615384615|                  15.750645708563392|   536384|
|   536387|             288.0|                  117.57550765359255|   536387|
|   536397|              30.0|                                18.0|   536397|
|   536405|             128.0|                                 0.0|   536405|
|   536407|               6.0|                                 0.0|   536407|
|   536463|              12.0|                                 0.0|   536463|
|   536500|               6.8|                   4.019950248448356|   536500|
|   536522|1.5925925925925926|                  1.60460585351366

# Window Functions

In [200]:
from pyspark.sql.functions import col,to_date
dfWithDate = df.withColumn("date",to_date(col("InvoiceDate"),"MM/d/yyyy H:mm"))
dfWithDate.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|      date|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|2010-12-01|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|2010-12-01|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|U

In [201]:
dfWithDate.createOrReplaceTempView("dfWithDate")

In [202]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc

windowSpec = Window\
.partitionBy("CustomerId", "date")\
.orderBy(desc("Quantity"))\
.rowsBetween(Window.unboundedPreceding, Window.currentRow)


In [203]:
from pyspark.sql.functions import max
maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)

In [204]:
from pyspark.sql.functions import dense_rank,rank
purchaseDenseRank = dense_rank().over(windowSpec)
purchaseRank = rank().over(windowSpec)

In [205]:
from pyspark.sql.functions import col
dfWithDate.where("CustomerId IS NOT NULL").orderBy("CustomerId")\
.select(
col("CustomerId"),
col("date"),
col("Quantity"),
purchaseRank.alias("quantityRank"),
purchaseDenseRank.alias("quantityDenseRank"),
maxPurchaseQuantity.alias("maxPurchaseQuantity")
).show()

+----------+----+--------+------------+-----------------+-------------------+
|CustomerId|date|Quantity|quantityRank|quantityDenseRank|maxPurchaseQuantity|
+----------+----+--------+------------+-----------------+-------------------+
|     12346|null|   74215|           1|                1|              74215|
|     12346|null|  -74215|           2|                2|              74215|
|     12347|null|     240|           1|                1|                240|
|     12347|null|      36|           2|                2|                240|
|     12347|null|      36|           2|                2|                240|
|     12347|null|      36|           2|                2|                240|
|     12347|null|      24|           5|                3|                240|
|     12347|null|      24|           5|                3|                240|
|     12347|null|      24|           5|                3|                240|
|     12347|null|      24|           5|                3|       

In [206]:
spark.sql("""SELECT 
CustomerId, 
date, 
Quantity,
rank(Quantity) OVER (PARTITION BY CustomerId, date ORDER BY Quantity DESC NULLS LAST ROWS BETWEEN UNBOUNDED PRECEDING AND
CURRENT ROW) as rank,
dense_rank(Quantity) OVER (PARTITION BY CustomerId, date ORDER BY Quantity DESC NULLS LAST ROWS BETWEEN UNBOUNDED PRECEDING AND
CURRENT ROW) as dRank,
max(Quantity) OVER (PARTITION BY CustomerId, date ORDER BY Quantity DESC NULLS LAST ROWS BETWEEN UNBOUNDED PRECEDING AND
CURRENT ROW) as maxPurchase
FROM 
    dfWithDate 
WHERE 
    CustomerId IS NOT NULL ORDER BY CustomerId""").show()

+----------+----+--------+----+-----+-----------+
|CustomerId|date|Quantity|rank|dRank|maxPurchase|
+----------+----+--------+----+-----+-----------+
|     12346|null|  -74215|   2|    2|      74215|
|     12346|null|   74215|   1|    1|      74215|
|     12347|null|      24|   5|    3|        240|
|     12347|null|      24|   5|    3|        240|
|     12347|null|      24|   5|    3|        240|
|     12347|null|      36|   2|    2|        240|
|     12347|null|      24|   5|    3|        240|
|     12347|null|      24|   5|    3|        240|
|     12347|null|      24|   5|    3|        240|
|     12347|null|     240|   1|    1|        240|
|     12347|null|      24|   5|    3|        240|
|     12347|null|      24|   5|    3|        240|
|     12347|null|      24|   5|    3|        240|
|     12347|null|      24|   5|    3|        240|
|     12347|null|      24|   5|    3|        240|
|     12347|null|      24|   5|    3|        240|
|     12347|null|      36|   2|    2|        240|


# Grouping Sets

In [207]:
"""https://www.waitingforcode.com/apache-spark-sql/grouping-sets-apache-spark-sql/read"""
dfNoNull = dfWithDate.drop()
dfNoNull.createOrReplaceTempView("dfNoNull")

In [208]:
spark.sql("""SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNull
GROUP BY customerId, stockCode
ORDER BY CustomerId DESC, stockCode DESC""").show()

+----------+---------+-------------+
|CustomerId|stockCode|sum(Quantity)|
+----------+---------+-------------+
|     18287|    85173|           48|
|     18287|   85040A|           48|
|     18287|   85039B|          120|
|     18287|   85039A|           96|
|     18287|    84920|            4|
|     18287|    84584|            6|
|     18287|   84507C|            6|
|     18287|   72351B|           24|
|     18287|   72351A|           24|
|     18287|   72349B|           60|
|     18287|    47422|           24|
|     18287|    47421|           48|
|     18287|    35967|           36|
|     18287|    23445|           20|
|     18287|    23378|           24|
|     18287|    23376|           48|
|     18287|    23310|           36|
|     18287|    23274|           12|
|     18287|    23272|           12|
|     18287|    23269|           36|
+----------+---------+-------------+
only showing top 20 rows



In [209]:
spark.sql("""SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNull
GROUP BY customerId, stockCode GROUPING SETS((customerId, stockCode))
ORDER BY CustomerId DESC, stockCode DESC""").show()

+----------+---------+-------------+
|customerId|stockCode|sum(Quantity)|
+----------+---------+-------------+
|     18287|    85173|           48|
|     18287|   85040A|           48|
|     18287|   85039B|          120|
|     18287|   85039A|           96|
|     18287|    84920|            4|
|     18287|    84584|            6|
|     18287|   84507C|            6|
|     18287|   72351B|           24|
|     18287|   72351A|           24|
|     18287|   72349B|           60|
|     18287|    47422|           24|
|     18287|    47421|           48|
|     18287|    35967|           36|
|     18287|    23445|           20|
|     18287|    23378|           24|
|     18287|    23376|           48|
|     18287|    23310|           36|
|     18287|    23274|           12|
|     18287|    23272|           12|
|     18287|    23269|           36|
+----------+---------+-------------+
only showing top 20 rows



# Rollups

In [210]:
"""A rollup is a multidimensional aggregation that performs a variety of group-by style calculations
for us."""
rolledUpDF = dfNoNull.rollup("Date","Country").agg(sum("Quantity"))\
.selectExpr("Date","Country","`sum(Quantity)` as total_quantity").orderBy("Date")

rolledUpDF.show(50)

+----------+--------------------+--------------+
|      Date|             Country|total_quantity|
+----------+--------------------+--------------+
|      null|              France|         71048|
|      null|             Belgium|         14594|
|      null|        Saudi Arabia|            75|
|      null|            Portugal|          9292|
|      null|             Austria|          3468|
|      null|             Lebanon|           386|
|      null|             Iceland|          1271|
|      null|     Channel Islands|          7219|
|      null|      United Kingdom|       2667928|
|      null|                null|       5176450|
|      null|         Netherlands|        139467|
|      null|         Unspecified|          2659|
|      null|             Germany|         76234|
|      null|              Norway|          8246|
|      null|           Hong Kong|          4388|
|      null|               Spain|         18918|
|      null|           Australia|         71593|
|      null|        

In [211]:
dfNoNull.where("date is null").show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|date|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----+
|   539993|    22386|JUMBO BAG PINK PO...|      10|1/4/2011 10:00|     1.95|     13313|United Kingdom|null|
|   539993|    21499|  BLUE POLKADOT WRAP|      25|1/4/2011 10:00|     0.42|     13313|United Kingdom|null|
|   539993|    21498| RED RETROSPOT WRAP |      25|1/4/2011 10:00|     0.42|     13313|United Kingdom|null|
|   539993|    22379|RECYCLING BAG RET...|       5|1/4/2011 10:00|      2.1|     13313|United Kingdom|null|
|   539993|    20718|RED RETROSPOT SHO...|      10|1/4/2011 10:00|     1.25|     13313|United Kingdom|null|
|   539993|   85099B|JUMBO BAG RED RET...|      10|1/4/2011 10:00|     1.95|     13313|United Kingdom|null|
|   539993|    20682|RED RET

In [212]:
rolledUpDF.where("Country is NULL").show()

+----------+-------+--------------+
|      Date|Country|total_quantity|
+----------+-------+--------------+
|      null|   null|       3297071|
|      null|   null|       5176450|
|2010-12-01|   null|         26814|
|2010-12-02|   null|         21023|
|2010-12-03|   null|         14830|
|2010-12-05|   null|         16395|
|2010-12-06|   null|         21419|
|2010-12-07|   null|         24995|
|2010-12-08|   null|         22741|
|2010-12-09|   null|         18431|
|2010-12-10|   null|         20297|
|2010-12-12|   null|         10565|
|2010-12-13|   null|         17623|
|2010-12-14|   null|         20098|
|2010-12-15|   null|         18229|
|2010-12-16|   null|         29632|
|2010-12-17|   null|         16069|
|2010-12-19|   null|          3795|
|2010-12-20|   null|         14965|
|2010-12-21|   null|         15467|
+----------+-------+--------------+
only showing top 20 rows



In [213]:
rolledUpDF.where("Date is null").show()

+----+--------------------+--------------+
|Date|             Country|total_quantity|
+----+--------------------+--------------+
|null|            Portugal|          9292|
|null|             Bahrain|           206|
|null|           Australia|         71593|
|null|               Spain|         18918|
|null|              Greece|          1320|
|null|           Singapore|          4635|
|null|              Norway|          8246|
|null|        Saudi Arabia|            75|
|null|              Canada|          2763|
|null|                null|       3297071|
|null|              Israel|          4310|
|null|              Poland|          2580|
|null|                 USA|           137|
|null|  European Community|           497|
|null|                null|       5176450|
|null|              France|         71048|
|null|               Japan|         15800|
|null|United Arab Emirates|           982|
|null|         Unspecified|          2659|
|null|              Sweden|         23810|
+----+-----

# Cube

In [214]:
"""https://www.mikulskibartosz.name/cube-and-rollup-in-apache-spark/"""
from pyspark.sql.functions import sum
dfNoNull.cube("Date","Country").agg(sum(col("Quantity")))\
.select("Date","Country","sum(Quantity)").orderBy("Date").show()

+----+--------------------+-------------+
|Date|             Country|sum(Quantity)|
+----+--------------------+-------------+
|null|              Norway|        19247|
|null|             Germany|        76234|
|null|           Lithuania|          652|
|null|              Sweden|        23810|
|null|               Italy|         4060|
|null|             Bahrain|          260|
|null|                null|      5176450|
|null|               Spain|        26824|
|null|               Italy|         7999|
|null|              Poland|         3653|
|null|United Arab Emirates|          982|
|null|        Saudi Arabia|           75|
|null|         Unspecified|         2659|
|null|              France|        71048|
|null|             Belgium|        23152|
|null|               Japan|        15800|
|null|     Channel Islands|         9479|
|null|           Singapore|         5234|
|null|             Finland|        10666|
|null|United Arab Emirates|          982|
+----+--------------------+-------

# Group MetaData

In [215]:
"""from pyspark.sql.functions import grouping_id,sum,expr

dfNoNull.cube("customerId", "stockCode").agg(grouping_id(),sum("Quantity")).orderBy(expr("grouping_id()").desc).show()"""

'from pyspark.sql.functions import grouping_id,sum,expr\n\ndfNoNull.cube("customerId", "stockCode").agg(grouping_id(),sum("Quantity")).orderBy(expr("grouping_id()").desc).show()'

# Pivot

In [216]:
pivoted = dfWithDate.groupBy("date").pivot("Country").sum()
pivoted.show()

+----------+---------------------------------------+------------------------+-----------------------------------------+-------------------------------------+----------------------+---------------------------------------+-------------------------------------+----------------------+---------------------------------------+-------------------------------------+----------------------+---------------------------------------+------------------------------------+---------------------+--------------------------------------+------------------------------------+---------------------+--------------------------------------+---------------------------------------------+------------------------------+-----------------------------------------------+------------------------------------+---------------------+--------------------------------------+--------------------------------------------+-----------------------------+----------------------------------------------+-------------------------------------

In [217]:
pivoted.where("date>'2011-12-05'").select("date","`USA_sum(CAST(Quantity AS BIGINT))`").show()

+----------+---------------------------------+
|      date|USA_sum(CAST(Quantity AS BIGINT))|
+----------+---------------------------------+
|2011-12-07|                             null|
|2011-12-06|                             null|
|2011-12-08|                             -196|
|2011-12-09|                             null|
+----------+---------------------------------+



# JOIN

In [218]:
person = spark.createDataFrame([(0, "Bill Chambers", 0, [100]),(1, "Matei Zaharia", 1, [500, 250, 100]),
(2, "Michael Armbrust", 1, [250, 100])]).toDF("id", "name", "graduate_program", "spark_status")

In [219]:
graduateProgram = spark.createDataFrame([
(0, "Masters", "School of Information", "UC Berkeley"),
(2, "Masters", "EECS", "UC Berkeley"),
(1, "Ph.D.", "EECS", "UC Berkeley")]).toDF("id", "degree", "department", "school")

In [220]:
sparkStatus = spark.createDataFrame([
(500, "Vice President"),
(250, "PMC Member"),
(100, "Contributor")]).toDF("id", "status")

In [221]:
# register as database tables by creating view
person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

Inner Join 

In [222]:
joinExpression = person["graduate_program"]==graduateProgram["id"]

In [223]:
person.join(graduateProgram,joinExpression).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [224]:
spark.sql("""SELECT * FROM person JOIN graduateProgram
ON person.graduate_program = graduateProgram.id""").show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [225]:
joinType="inner"

person.join(graduateProgram,joinExpression,joinType).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



OUTER JOIN

In [226]:
joinType="outer"

person.join(graduateProgram,joinExpression,joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [227]:
joinType="left_outer"

graduateProgram.join(person,joinExpression,joinType).show()

+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
| id| degree|          department|     school|  id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|   0|   Bill Chambers|               0|          [100]|
|  2|Masters|                EECS|UC Berkeley|null|            null|            null|           null|
|  1|  Ph.D.|                EECS|UC Berkeley|   1|   Matei Zaharia|               1|[500, 250, 100]|
|  1|  Ph.D.|                EECS|UC Berkeley|   2|Michael Armbrust|               1|     [250, 100]|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+



In [228]:
spark.sql("""SELECT * FROM graduateProgram LEFT OUTER JOIN person
ON person.graduate_program = graduateProgram.id""").show()

+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
| id| degree|          department|     school|  id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|   0|   Bill Chambers|               0|          [100]|
|  2|Masters|                EECS|UC Berkeley|null|            null|            null|           null|
|  1|  Ph.D.|                EECS|UC Berkeley|   1|   Matei Zaharia|               1|[500, 250, 100]|
|  1|  Ph.D.|                EECS|UC Berkeley|   2|Michael Armbrust|               1|     [250, 100]|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+



In [229]:
joinType = "right_outer"
person.join(graduateProgram, joinExpression, joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [230]:
spark.sql("""SELECT * FROM person RIGHT OUTER JOIN graduateProgram
ON person.graduate_program = graduateProgram.id""").show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [231]:
joinType="left_semi"

graduateProgram.join(person,joinExpression,joinType).show()

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+



In [232]:
gradProgram2 = graduateProgram.union(spark.createDataFrame([(0, "Masters", "Duplicated Row", "Duplicated School")]))

In [233]:
gradProgram2.createOrReplaceTempView("gradProgram2")

In [234]:
gradProgram2.join(person,joinExpression,joinType).show()

+---+-------+--------------------+-----------------+
| id| degree|          department|           school|
+---+-------+--------------------+-----------------+
|  0|Masters|School of Informa...|      UC Berkeley|
|  1|  Ph.D.|                EECS|      UC Berkeley|
|  0|Masters|      Duplicated Row|Duplicated School|
+---+-------+--------------------+-----------------+



In [235]:
gradProgram2.createOrReplaceTempView("gradProgram2")

In [236]:
gradProgram2.join(person,joinExpression,joinType).show()

+---+-------+--------------------+-----------------+
| id| degree|          department|           school|
+---+-------+--------------------+-----------------+
|  0|Masters|School of Informa...|      UC Berkeley|
|  1|  Ph.D.|                EECS|      UC Berkeley|
|  0|Masters|      Duplicated Row|Duplicated School|
+---+-------+--------------------+-----------------+



In [237]:
spark.sql("""SELECT * FROM gradProgram2 LEFT SEMI JOIN person
ON gradProgram2.id = person.graduate_program""").show()

+---+-------+--------------------+-----------------+
| id| degree|          department|           school|
+---+-------+--------------------+-----------------+
|  0|Masters|School of Informa...|      UC Berkeley|
|  1|  Ph.D.|                EECS|      UC Berkeley|
|  0|Masters|      Duplicated Row|Duplicated School|
+---+-------+--------------------+-----------------+



LEFT ANTI JOIN

In [238]:
joinType="left_anti"
graduateProgram.join(person,joinExpression,joinType).show()

+---+-------+----------+-----------+
| id| degree|department|     school|
+---+-------+----------+-----------+
|  2|Masters|      EECS|UC Berkeley|
+---+-------+----------+-----------+



In [239]:
spark.sql("""SELECT * FROM graduateProgram LEFT ANTI JOIN person
ON graduateProgram.id = person.graduate_program""").show()

+---+-------+----------+-----------+
| id| degree|department|     school|
+---+-------+----------+-----------+
|  2|Masters|      EECS|UC Berkeley|
+---+-------+----------+-----------+



CROSS JOIN

In [240]:
joinType="cross"
graduateProgram.join(person,joinExpression,joinType).show()

+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
| id| degree|          department|     school| id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|  0|   Bill Chambers|               0|          [100]|
|  1|  Ph.D.|                EECS|UC Berkeley|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  1|  Ph.D.|                EECS|UC Berkeley|  2|Michael Armbrust|               1|     [250, 100]|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+



In [241]:
spark.sql("""SELECT * FROM graduateProgram CROSS JOIN person
ON graduateProgram.id = person.graduate_program
""").show()

+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
| id| degree|          department|     school| id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|  0|   Bill Chambers|               0|          [100]|
|  1|  Ph.D.|                EECS|UC Berkeley|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  1|  Ph.D.|                EECS|UC Berkeley|  2|Michael Armbrust|               1|     [250, 100]|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+



In [242]:
person.crossJoin(graduateProgram).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  0|   Bill Chambers|               0|          [100]|  2|Masters|                EECS|UC Berkeley|
|  0|   Bill Chambers|               0|          [100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  2|Masters|                EECS|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  0|Masters|School of Informa...|UC 

In [243]:
from pyspark.sql.functions import expr
person.withColumnRenamed("id","personId").join(sparkStatus,expr("array_contains(spark_status,id)")).show()

+--------+----------------+----------------+---------------+---+--------------+
|personId|            name|graduate_program|   spark_status| id|        status|
+--------+----------------+----------------+---------------+---+--------------+
|       0|   Bill Chambers|               0|          [100]|100|   Contributor|
|       1|   Matei Zaharia|               1|[500, 250, 100]|500|Vice President|
|       1|   Matei Zaharia|               1|[500, 250, 100]|250|    PMC Member|
|       1|   Matei Zaharia|               1|[500, 250, 100]|100|   Contributor|
|       2|Michael Armbrust|               1|     [250, 100]|250|    PMC Member|
|       2|Michael Armbrust|               1|     [250, 100]|100|   Contributor|
+--------+----------------+----------------+---------------+---+--------------+



In [244]:
spark.sql("""SELECT * FROM
(select id as personId, name, graduate_program, spark_status FROM person)
INNER JOIN sparkStatus ON array_contains(spark_status, id)""").show()

+--------+----------------+----------------+---------------+---+--------------+
|personId|            name|graduate_program|   spark_status| id|        status|
+--------+----------------+----------------+---------------+---+--------------+
|       0|   Bill Chambers|               0|          [100]|100|   Contributor|
|       1|   Matei Zaharia|               1|[500, 250, 100]|500|Vice President|
|       1|   Matei Zaharia|               1|[500, 250, 100]|250|    PMC Member|
|       1|   Matei Zaharia|               1|[500, 250, 100]|100|   Contributor|
|       2|Michael Armbrust|               1|     [250, 100]|250|    PMC Member|
|       2|Michael Armbrust|               1|     [250, 100]|100|   Contributor|
+--------+----------------+----------------+---------------+---+--------------+



# READ/WRITE Files

CSV

In [245]:
csvFile=spark.read.format("csv")\
.option("header","true")\
.option("mode","FAILFAST")\
.option("inferSchema","true")\
.load(r"C:\Users\Admin\Documents\spark\Spark-The-Definitive-Guide-master\data\flight-data\csv\2010-summary.csv")

In [246]:
csvFile.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



WRITE FILE

In [247]:
csvFile.write.format("csv").mode("overwrite").option("sep","\t").save(r"D:\sparkbookwritedemo\my-tsv-file.tsv")

JSON

In [248]:
spark.read.format("json").option("mode","FAILFAST").option("inferSchema","true")\
.load("C:/Users/Admin/Documents/spark/Spark-The-Definitive-Guide-master/data/flight-data/json/2010-summary.json").show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

WRITING JSON

In [249]:
Path="D:/sparkbookwritedemo/"
csvFile.write.format("json").mode("overwrite").save(Path+"my-json-file.json")

Parquet

In [250]:
path="C:/Users/Admin/Documents/spark/Spark-The-Definitive-Guide-master/data/"
spark.read.format("parquet").load(path+"flight-data/parquet/2010-summary.parquet").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [251]:
csvFile.write.format("parquet").mode("overwrite").save(Path+"my-parquet-file.parquet")

ORC Files

In [252]:
spark.read.format("orc").load(path+"flight-data/orc/2010-summary.orc").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [253]:
csvFile.write.format("orc").mode("overwrite").save(Path+"my-json-file.orc")

Text Files

In [254]:
spark.read.text(path+"flight-data/csv/2010-summary.csv").selectExpr("split(value,',') as row").show()

+--------------------+
|                 row|
+--------------------+
|[DEST_COUNTRY_NAM...|
|[United States, R...|
|[United States, I...|
|[United States, I...|
|[Egypt, United St...|
|[Equatorial Guine...|
|[United States, S...|
|[United States, G...|
|[Costa Rica, Unit...|
|[Senegal, United ...|
|[United States, M...|
|[Guyana, United S...|
|[United States, S...|
|[Malta, United St...|
|[Bolivia, United ...|
|[Anguilla, United...|
|[Turks and Caicos...|
|[United States, A...|
|[Saint Vincent an...|
|[Italy, United St...|
+--------------------+
only showing top 20 rows



In [255]:
csvFile.select("DEST_COUNTRY_NAME").write.mode("overwrite").text(Path+"/simple-text-file.txt")

In [256]:
csvFile.limit(10).select("DEST_COUNTRY_NAME", "count")\
.write.mode("overwrite").partitionBy("count").text(Path+"five-csv-files2py.csv")

Bucket

In [257]:
numberBuckets=10
columnToBucketBy="count"

#csvFile.write.format("parquet").mode("overwrite").bucketBy(numberBuckets,columnToBucketBy).saveAsTable("bucketedFiles")

# SPARK SQL

In [258]:
spark.read.json(path+"flight-data/json/2015-summary.json").createOrReplaceTempView("some_sql_view")

In [259]:
# you can use spark sql and dataframe function simultaneously
spark.sql("""
    SELECT DEST_COUNTRY_NAME, sum(count)
FROM some_sql_view GROUP BY DEST_COUNTRY_NAME
""").where("DEST_COUNTRY_NAME like 'S%'").where("`sum(count)` > 10").count()

12

Creating Tables

In [260]:
spark.sql("""
    CREATE TABLE flights (
        DEST_COUNTRY_NAME STRING,
        ORIGIN_COUNTRY_NAME STRING,
        count LONG)
        USING JSON OPTIONS 
        (path 'C:/Users/Admin/Documents/spark/Spark-The-Definitive-Guide-master/data/flight-data/json/2015-summary.json')
""").show()

++
||
++
++



In [261]:
spark.sql("select * from flights").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [262]:
spark.sql("""
CREATE TABLE flights_csv (
    DEST_COUNTRY_NAME STRING,
    ORIGIN_COUNTRY_NAME STRING COMMENT "remember, the US will be most prevalent",
    count LONG)
USING csv OPTIONS 
(header true, path 'C:/Users/Admin/Documents/spark/Spark-The-Definitive-Guide-master/data/flight-data/csv/2015-summary.csv')
""")

DataFrame[]

In [263]:
spark.sql("select * from flights_csv").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [269]:
"""In this example, we are creating a Hive-compatible table because we did not explicitly specify the
format via USING"""

spark.sql("""
CREATE TABLE flights_from_select USING parquet AS SELECT * FROM flights""")

DataFrame[]

In [270]:
spark.sql("""CREATE TABLE IF NOT EXISTS flights_from_select
AS SELECT * FROM flights""")

DataFrame[]

In [271]:
spark.sql("""select * from flights_from_select""").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [274]:
"""you can control the layout of the data by writing out a partitioned dataset"""

spark.sql("""CREATE TABLE partitioned_flights USING parquet PARTITIONED BY (DEST_COUNTRY_NAME)
AS SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count FROM flights LIMIT 5""")

DataFrame[]

In [275]:
spark.sql("select * from partitioned_flights").show(3)

+-------------------+-----+-----------------+
|ORIGIN_COUNTRY_NAME|count|DEST_COUNTRY_NAME|
+-------------------+-----+-----------------+
|      United States|   15|            Egypt|
|            Romania|   15|    United States|
|            Croatia|    1|    United States|
+-------------------+-----+-----------------+
only showing top 3 rows



In [279]:
import re
re.match(r".*?hello", "xhelloxxxxxx xhelloxxxxxx xhelloxxxxxx")

<re.Match object; span=(0, 6), match='xhello'>

In [282]:
pets = re.findall(r'([A-Za-z]+)\s\w+\s(\d+)\s(\w+)', "Clary has 2 dogs but John has 3 cats")
pets[1][0]

'John'